# 스테이블디퓨전으로_이미지생성서비스_만들기
* 간단한 스케치를 기반으로 이미지 생성
* 스케치가 되어있는 이미지를 업로드해서 생성

In [2]:
# !pip install diffusers

In [3]:
# !pip install ipywidgets

In [4]:
import os
from typing import IO
import gradio as gr
import requests
import torch
from tqdm import tqdm
from PIL import Image
from diffusers import StableDiffusionImg2ImgPipeline

# 스케치 투 이미지 생성 UI 구현하기

In [2]:
WIDTH = 512
HEIGHT = 512

with gr.Blocks() as app:
    gr.Markdown("## 프롬프트 입력")
    with gr.Row():
        prompt = gr.Textbox(label="Prompt")
    with gr.Row():
        negative_prompt = gr.Textbox(label="Negative Prompt")
        
    gr.Markdown("## 스케치 to 이미지 생성")
    with gr.Row():
        with gr.Column():
            with gr.Tab("Canvas"):
                with gr.Row():
                    canvas = gr.Image(
                        label = "Draw",
                        source = 'canvas',
                        image_mode = 'RGB',
                        tool = 'color-sketch',
                        interactive = True,  # Corrected here
                        width = WIDTH,
                        height = HEIGHT,
                        shape = (WIDTH, HEIGHT),
                        brush_radius = 20,
                        type = 'pil'
                    )
               
                with gr.Row():
                    canvas_run_btn = gr.Button(value="Generate")
            
            with gr.Tab("File"):
                with gr.Row():
                    file = gr.Image(
                        label = "Upload",
                        source = 'upload',
                        image_mode = 'RGB',
                        tool = 'color-sketch',
                        interactive = True,  # Corrected here
                        width = WIDTH,
                        height = HEIGHT,
                        shape = (WIDTH, HEIGHT),
                        type = 'pil'
                    )
                with gr.Row():
                    file_run_btn = gr.Button(value="Generate")


IMPORTANT: You are using gradio version 3.40.0, however version 4.29.0 is available, please upgrade.
--------


In [4]:
app.launch(inline=False, share=True)

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
Running on public URL: https://1ccb199e5d9478c378.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [5]:
app.close()

Closing server running on port: 7860


# 모델 다운로드 UI 구현하기

In [12]:
with gr.Blocks() as app:
    gr.Markdown("## 모델 다운로드")
    with gr.Row():
        model_url = gr.Textbox(label = "모델 URL", placeholder = "https://civitai.com/")
        download_model_btn = gr.Button(value = "모델 다운로드")
    with gr.Row():
        model_file = gr.File(label = "모델 파일")
        
    download_model_btn.click(
        download_model,
        [model_url],
        [model_file]
    )

IMPORTANT: You are using gradio version 3.40.0, however version 4.29.0 is available, please upgrade.
--------


In [13]:
app.launch(inline=False, share=True)

Running on local URL:  http://127.0.0.1:7862
Running on public URL: https://5573f05e9ea6ad8e29.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


models/disneyPixarCartoon_v10.safetensors: 100%|██████████████████████| 3.95G/3.95G [10:59<00:00, 6.44MiB/s]


[INFO] File downloaded: models/disneyPixarCartoon_v10.safetensors


In [19]:
app.close()

Closing server running on port: 7862


# 모델 다운로드 기능 구현하기

In [11]:
import os
import glob

# 전역 변수로 모델 경로와 파일명을 저장
MODEL_PATH = None

#URL로부터 파일 다운로드 함수
def download_from_url(url, file_path, chunk_size=1024):
    try:
        resp = requests.get(url, stream=True)
        resp.raise_for_status()
    except Exception as e:
        print(f"[Error] {e}")
        raise e
        
    total = int(resp.headers.get('content-length', 0)) # 파일 크기 추출
    with open(file_path, 'wb') as file, tqdm(desc = file_path, total=total, unit = 'iB', unit_scale=True, 
                                            unit_divisor=1024) as bar:
        for data in resp.iter_content(chunk_size = chunk_size):
            size = file.write(data)
            bar.update(size)
            

# 모델을 다운로드하고 경로를 기억하는 함수
def download_model(url: str) -> str :
    global MODEL_PATH # 전역변수 사용해서 경로를 기억
    
    model_id = url.replace("https://civitai.com/models/", "").split("/")[0]
    
    try:
        response = requests.get(f"https://civitai.com/api/v1/models/{model_id}", timeout = 6000)
    except Exception as e:
        print(f"[Error] {e}")
        raise e
        
    download_url = response.json()['modelVersions'][0]['downloadUrl']
    filename = response.json()['modelVersions'][0]['files'][0]['name']
    
    file_path = f"models/{filename}"
    if os.path.exists(file_path):
        print(f"[INFO] File already exists : {file_path}")
        MODEL_PATH = file_path # 모델 경로 기억
        return file_path
    
    os.makedirs("models", exist_ok=True)
    download_from_url(download_url, file_path)
    print(f"[INFO] File downloaded: {file_path}")
    
    
    # 모델 경로 기억
    MODEL_PATH = file_path
    return file_path
       
# ./models 폴더에서 가장 최근에 수정된 모델 파일 찾기
def find_latest_model_in_directory(directory):
    model_files = glob.glob(f"{directory}/*.safetensors")
    if not model_files:
        return None
    
    # 가장 최근에 수정된 모델 파일 선택
    latest_model = max(model_files, key=os.path.getmtime)
    return latest_model
    
    
    

# 다운로드한 모델 불러와서 초기화하기

In [14]:
def init_pipeline():
    global MODEL_PATH
    
    if MODEL_PATH is None:
        MODEL_PATH = find_latest_model_in_directory("./models/")
    if MODEL_PATH is None:
        return "Error : No model found in ./models"
    
    global PIPELINE
    
    try:
        PIPELINE = StableDiffusionImg2ImgPipeline.from_single_file(
            MODEL_PATH,
            torch_dtype=torch.float16,
            variant="fp16",
            use_safetensors=True,
        ).to('cpu')
        print("[INFO] Initialized pipeline")
        return "Model Loaded"
    
    except Exception as e:
        print(f"[Error] {e}")

In [15]:
with gr.Blocks() as app:
    gr.Markdown("## 모델 불러오기")
    with gr.Row():
        load_model_btn = gr.Button(value="모델 불러오기")
    with gr.Row():
        is_model_check = gr.Textbox(label = "Model Load Check", value="Model Not Loaded")
        
    load_model_btn.click(
        init_pipeline,
        None,
        [is_model_check]
    )

IMPORTANT: You are using gradio version 3.40.0, however version 4.29.0 is available, please upgrade.
--------


In [16]:
app.queue().launch(inline=False, share=True)

Running on local URL:  http://127.0.0.1:7863
Running on public URL: https://935b01a7a2607bb2fc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Loading pipeline components...:   0%|                                                 | 0/6 [00:00<?, ?it/s]/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
Some weights of the model checkpoint were not used when initializing CLIPTextModel: 
 ['text_model.embeddings.position_ids']
Loading pipeline components...: 100%|█████████████████████████████████████████| 6/6 [00:34<00:00,  5.77s/it]
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion_img2img.StableDiffusionImg2ImgPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hu

[INFO] Initialized pipeline


In [1]:
app.close()

NameError: name 'app' is not defined

# 스케치 투 이미지 생성기능 구현

In [ ]:
def sketch_to_image(sketch, prompt, negative_prompt):
    global PIPELINE
    if PIPELINE is None:
        return "Error Pipeline is not initialized"
    
    prompt = [prompt]
    negative_prompt = [negative_prompt]
    
    images = [sketch]* len(prompt)
    
    try:
        # 이미지 생성
        result = PIPELINE(
            image = images,
            prompt = prompt,
            negative_prompt = negative_prompt,
            height = height,
            width = width,
            num_images_per_prompt = 4,
            num_inference_steps = 10,
            strength=0.7
        ).images
        
    except Exception as e:
        print(e)
        
#     # GPU 메모리 캐시 비우기
#     with torch.cuda.device("cuda")
#         torch.cuda.empty_chache()

    return result

# 이미지 생성 전체 코드

In [26]:
import os
import gradio as gr
import requests
import torch
import glob

from typing import IO
from tqdm import tqdm
from PIL import Image
from diffusers import StableDiffusionImg2ImgPipeline

In [27]:
WIDTH = 512
HEIGHT = 512

# 전역 변수로 모델 경로와 파일명을 저장
MODEL_PATH = None
PIPELINE = None

In [28]:



# URL로부터 파일 다운로드 함수
def download_from_url(url, file_path, chunk_size=1024):
    try:
        resp = requests.get(url, stream = True)
        resp.raise_for_status()
    except Exception as e:
        print(f"[Error] {e}")
        raise e

    total = int(resp.headers.get('content-length', 0))  # 파일 크기 추출
    with open(file_path, 'wb') as file, tqdm(desc=file_path, total=total, unit='iB', unit_scale=True,
                                            unit_divisor = 1024) as bar:
        for data in resp.iter_content(chunk_size=chunk_size):
            size = file.write(data)
            bar.update(size)


# 모델을 다운로드하고 경로를 기억하는 함수
def download_model(url: str) -> str:
    global MODEL_PATH    # 전역 변수를 사용해서 경로를 파악

    model_id = url.replace("https://civitai.com/models/", "").split("/")[0]

    try:
        response = requests.get(f"https://civitai.com/api/v1/models/{model_id}", timeout = 6000)
    except Exception as e:
        print(f"[Error] {e}")
        raise e

    download_url = response.json()['modelVersions'][0]['downloadUrl']
    filename = response.json()['modelVersions'][0]['files'][0]['name']

    file_path = f"models/{filename}"
    if os.path.exists(file_path):
        print(f"[INFO] File already exists: {file_path}")
        MODEL_PATH = file_path   # 모델 경로 기억
        return file_path

    os.makedirs("models", exist_ok = True)
    download_from_url(download_url, file_path)
    print(f"[INFO] File downloaded: {file_path}")

# 모델 경로 기억
    MODEL_PATH = file_path
    return file_path


# ./models 폴더에서 가장 최근에 수정된 모델 파일 찾기
def find_latest_model_in_directory(directory):
    model_files = glob.glob(f"{directory}/*.safetensors")
    if not model_files:
        return None


# 가장 최근에 수정된 모델 파일 선택
    latest_model = max(model_files, key =os.path.getmtime)
    return latest_model


def init_pipeline():
    global MODEL_PATH

    if MODEL_PATH is None:
        MODEL_PATH = find_latest_model_in_directory("./models")
    if MODEL_PATH is None:
        return "Error: No model found in ./models"

    global PIPELINE

    try:
        PIPELINE = StableDiffusionImg2ImgPipeline.from_single_file(
                MODEL_PATH,
                torch_dtype = torch.float16,
                variant = "fp16",
                use_safetensors = True,
        ).to('cuda')
        print("[INFO] Initialized pipeline")
        return "Model Loaded!"
    except Exception as e:
        print(f"[Error] {e}")

#모델을 불러와서 모델에서 이미지 생성하는
def sketch_to_image(sketch, prompt, negative_prompt):
    global PIPELINE
    if PIPELINE is None:
        return "Error Pipeline is not initialized"
    prompt = [prompt]
    negative_prompt =[negative_prompt]

    images =[sketch] * len(prompt)

    try:
        #이미지 생성
        result = PIPELINE(
            image=images,
            prompt=prompt,
            negative_prompt= negative_prompt,
            height= height,
            width=width,
            num_images_per_prompt=4,
            num_inference_steps=20,
            strength = 0.7
        ).images
    except Excption as e:
        print(e)
        return e
        #gpu메모리 캐시 비우기
        #with torch.cuda.device("cuda")
        #torch.cuda.empty_cache()

    return result

In [29]:
with gr.Blocks() as app:

    # 모델 다운로드 블록
    gr.Markdown("## 모델 다운로드")
    with gr.Row():
        model_url = gr.Textbox(label = "모델 URL", placeholder = "https://civitai.com/")
        download_model_btn = gr.Button(value = "모델 다운로드")
    with gr.Row():
        model_file = gr.File(label = "모델 파일")


    # 모델 불러오기 블록
    gr.Markdown("## 모델 불러오기")
    with gr.Row():
        load_model_btn = gr.Button(value="모델 불러오기")
    with gr.Row():
        is_model_check = gr.Textbox(label="Model Load Check", value="Model Not Loaded")


    # 프롬프트 입력 블록
    gr.Markdown("## 프롬프트 입력")
    with gr.Row():
        prompt = gr.Textbox(label="Prompt")
    with gr.Row():
        n_prompt = gr.Textbox(label="Negative Prompt")


    # 스케치에서 이미지 생성 블록
    gr.Markdown("## 스케치 to 이미지 생성")
    with gr.Row():
        with gr.Column():
            with gr.Tab("Canvas"):
                with gr.Row():
                    canvas = gr.Image(
                        label = "Draw",
                        source = "canvas",
                        image_mode = 'RGB',
                        tool = 'color-sketch',
                        interactive = True,
                        width = WIDTH,
                        height = HEIGHT,
                        shape = (WIDTH, HEIGHT),
                        brush_radius = 20,
                        type = 'pil'
                    )
                with gr.Row():
                    canvas_run_btn = gr.Button(value = "Generate")

            with gr.Tab("File"):
                with gr.Row():
                    file = gr.Image(
                        label = "Upload",
                        source = "upload",
                        image_mode = 'RGB',
                        tool = 'color-sketch',
                        interactive = True,
                        width = WIDTH,
                        height = HEIGHT,
                        shape = (WIDTH, HEIGHT),
                        type = 'pil'
                    )
                with gr.Row():
                    file_run_btn = gr.Button(value = "Generate")

        # 결과 이미지 갤러리
        with gr.Column():
            result_gallery = gr.Gallery(label="Output", height=512)


    # 모델 다운로드 실행
    download_model_btn.click(
        download_model,
        [model_url],
        [model_file]
        )
    # 모델 로드 실행
    load_model_btn.click(
        init_pipeline,
        None,
        [is_model_check]
        )
    # Canvas 에서 이미지 생성 버튼 실행
    canvas_run_btn.click(
        sketch_to_image,
        [canvas, prompt, n_prompt],
        [result_gallery]
        )
    # File 업로드에서 이미지 생성 버튼 실행
    file_run_btn.click(
        sketch_to_image,
        [file, prompt, n_prompt],
        [result_gallery]
        )




app.queue().launch(inline=False, share=True)


Running on local URL:  http://127.0.0.1:7864
IMPORTANT: You are using gradio version 3.40.0, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://48425f5271ea95e817.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/gradio/routes.py", line 488, in run_predict
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/gradio/blocks.py", line 1427, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/gradio/blocks.py", line 1109, in call_function
    prediction = await anyio.to_thread.run_sync(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/anyio/_backends/_asyncio.py", line 